_____
***

# PheKnowLator: Phenotype Knowledge Translator  
***
***

**Author:** [TJCallahan](https://mail.google.com/mail/u/0/?view=cm&fs=1&tf=1&to=callahantiff@gmail.com)  
**GitHub Repository:** [PheKnowLator](https://github.com/callahantiff/PheKnowLator/wiki)  
**Current Release:** **[`v2.0.0`](https://github.com/callahantiff/PheKnowLator/wiki/v2.0.0)**

<br>

**Objective:** Knowledge graphs provide meaningful ways to integrate heterogeneous biological data and represent complex biological mechanisms. This work seeks to explore the utility of incorporating existing knowledge of molecular mechanism from ontologies, publicly available data, and the literature to construct a biomedical knowledge graph that models unbiased molecular mechanisms of human disease.

<img src="https://user-images.githubusercontent.com/8030363/76589824-f3787180-64b0-11ea-8a07-6aca98ee6347.png" width="1200" height="1200">

<br>

***
***

## Notebook Purpose
**Wiki Page:** **[`Release v2.0.0`](https://github.com/callahantiff/PheKnowLator/wiki/v2.0.0)**

<br>

**Purpose:** This notebook serves as a `main` file for the PheKnowLator project. This scripts walks through this program step-by-step and generates the knowledge graph shown above. There is also a command line version of this file ([`Main.py`](https://github.com/callahantiff/PheKnowLator/blob/master/Main.py)). Please see the [README](https://github.com/callahantiff/PheKnowLator/blob/master/README.md) for more information.

<br>

**Assumptions:**
1. Please download the [OWLTools](https://github.com/owlcollab/owltools) library to the `./resources/lib` directory    
2. Make sure that the following input documents have been constructed (see the [Dependencies Wiki](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies) for more information):  
  - [`resource_info.txt`](https://github.com/callahantiff/PheKnowLator/blob/master/resources/resource_info.txt)
  - [`ontology_source_list.txt`](https://github.com/callahantiff/PheKnowLator/blob/master/resources/ontology_source_list.txt)
  - [`edge_source_list.txt`](https://github.com/callahantiff/PheKnowLator/blob/master/resources/edge_source_list.txt)   

3. Prepare [relations](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#relations-data) and [node metadata](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#node-metadata) files prior to running the scripts.  

4. Select a knowledge graph build type (for more information, see below)  

<br>

**Selecting a Build Type:**  
The knowledge graph build algorithm has been designed to run from three different stages of development: `full`, `partial`, and `post-closure`.

Build Type | Description | Use Cases  
:--: | -- | --   
`full` | Runs all build steps in the algorithm | You want to build a knowledge graph and will not use a reasoner  
`partial` | Runs all of the build steps in the algorithm through adding the edges<br><br> If `node_data` is provided, it will not be added to the knowledge graph, but instead used to filter the edges such that only those edges with valid node metadata are added to the knowledge graph<br><br> Node metadata can always be added to a `partial` built knowledge graph by running the build as `post-closure` | You want to build a knowledge graph and plan to run a reasoner over it<br><br> You want to build a knowledge graph, but do not want to include node metadata, filter OWL semantics, or generate triple lists  
`post-closure` | Assumes that a reasoner was run over a knowledge graph and that the remaining build steps should be applied to a closed knowledge graph. The remaining build steps include determining whether OWL semantics should be filtered and creating and writing triple lists | You have run the `partial` build, ran a reasoner over it, and now want to complete the algorithm<br><br> You want to use the algorithm to process metadata and owl semantics for an externally built knowledge graph

<br>

***
### Table of Contents
***
The three primary steps involved in building a knowledge graph are `Downloading Data Sources`, `Creating Edge Lists`, and `Building the knowledge graphs`.

* [Data Sources](#data-source)  
* [Create Edge Lists](#create-edges)  
* [Build Knowledge Graph](#build-kg)  

***

***

_____
### Set-Up Environment

In [2]:
# import needed libraries
import glob
import json
import pandas
import time

# import module
from pkt import downloads, edge_list, knowledge_graph

***
## Download Data Sources <a class="anchor" id="data-source"></a>
***

**Wiki Page:** **[`Dependencies`](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies)**  

**Purpose:**
This portion of this portion of the algorithm is to download:
1. [Download Ontology Data](#download-ontology-data)  
2. [Download Edge Data](#download-edge-data)   

<br>

**Input Files:**
- [`resource_info.txt`](https://www.dropbox.com/s/8pu6bbxpqrui8rq/resource_info.txt?dl=1)   
- [`ontology_source_list.txt`](https://www.dropbox.com/s/bmmaavyd499d7px/ontology_source_list.txt?dl=1)  
- [`edge_source_list.txt`](https://www.dropbox.com/s/mshkscxqzrp9u8a/edge_source_list.txt?dl=1) 

<br>

**Assumptions:**  
All sources used to construct our knowledge graph need to be preprocessed and ready to download prior to running this code. All mapping, filtering, and label data have been generated. For assistance with creating these datasets, see the [`Data_Preparation.ipynb`](https://github.com/callahantiff/PheKnowLator/blob/master/Data_Preparation.ipynb) Jupyter Notebook.


<br>

***
***
### Ontology Data  <a class="anchor" id="download-ontology-data"></a>
Ontologies are the core data structure used when building PheKnowLator. In the figure above, ontology data are shown in yellow boxes.

In [ ]:
ont = pkt.OntData('resources/ontology_source_list.txt')
ont.downloads_data_from_url('imports')
ont.writes_source_metadata_locally()

In [ ]:
# start = time.time()

# ont = pkt.OntData('resources/ontology_source_list.txt')
# ont.downloads_data_from_url('imports')
# ont.writes_source_metadata_locally()

# end = time.time()
# print('Total time to run code: {} seconds'.format(end - start))

<br>

### Edge Data   <a class="anchor" id="download-edge-data"></a>
In PheKnowLator, classes are nodes that originate from ontologies. Class data sources are Linked Data sources that are used to create edges in the knowledge graph and thus can connect to other class data sources. Sometimes we want to add data that is not already part of an ontology. In that case, data either be added as an `instance` of an existing ontology class or as its own `owl:class` by being added to the knowledge graph as a `subclass` of an existing `owl:class`.

In [ ]:
edges = pkt.LinkedData('resources/edge_source_list.txt')
edges.downloads_data_from_url('')
# edges.writes_source_metadata_locally()

In [ ]:
# start = time.time()

# edges = pkt.LinkedData('resources/edge_source_list.txt')
# edges.downloads_data_from_url('')
# edges.writes_source_metadata_locally()

# end = time.time()
# print('Total time to run code: {} seconds'.format(end - start))

<br>

***
***

## Create Edge Lists <a class="anchor" id="create-edges"></a>

**Wiki Page:** **[`Data Sources`](https://github.com/callahantiff/PheKnowLator/wiki/v2-Data-Sources)**

<br>

**Purpose:** The code below will take the dictionaries of processed data described above and use them to create edge lists for each of the edge types specified in the [`resource_info.txt`](https://github.com/callahantiff/PheKnowLator/blob/master/resources/resource_info.txt). Each edge list will be appended to a nested dictionary (see details below).

<br>

**Assumptions:**  
1. All `ontology` and `edge` data sources have been downloaded.   

2. All code in the [`Data_Preparation.ipynb`](https://github.com/callahantiff/PheKnowLator/blob/master/Data_Preparation.ipynb) Jupyter Notebook has been run. This Notebook contains code needed to generate all mapping, filtering, and label data.

<br>

**Output:** [`Master_Edge_List_Dict.json`](https://www.dropbox.com/s/4j0vrwx26dh8hd1/Master_Edge_List_Dict.json?dl=1). Below is an example of what the `Master Edge Dictionary` contains for each processed resource:  
```python
master_edges = {'chemical-disease'  :
                {'source_labels'    : ';MESH_;',
                 'data_type'        : 'class-class',
                 'edge_relation'    : 'RO_0002606',
                 'uri'              : ('http://purl.obolibrary.org/obo/',
                                       'http://purl.obolibrary.org/obo/'),
                 'row_splitter'     : '#',
                 'col_splitter'     : 't',
                 'column_indicies'  : '1;4',
                 'identifier_maps'  : '0:./MESH_CHEBI_MAP.txt;1:disease-dbxref-map',
                 'evidence_criteria': "5;!=;' ",
                 'filter_criteria'  : 'None',
                 'edge_list'        : ['...']}
```

***

In [ ]:
# combine data sources
combined_edges = dict(dict(cls.data_files, **inst.data_files), **ont.data_files)

# initialize edge dictionary class
master_edges = pkt.CreatesEdgeList(combined_edges, './resources/resource_info.txt')
master_edges.creates_knowledge_graph_edges()

In [ ]:
# start = time.time()

# # combine data sources
# combined_edges = dict(dict(cls.data_files, **inst.data_files), **ont.data_files)

# # initialize edge dictionary class
# master_edges = pkt.CreatesEdgeList(combined_edges, './resources/resource_info.txt')
# master_edges.creates_knowledge_graph_edges()

# end = time.time()
# print('Total time to run code: {} seconds'.format(end - start))

**Preview Master Edge Data**  
Generate a table that includes each `edge-type`, its primary `relation`, example identifiers, and count of unique edges.

In [ ]:
# # read in master edge dictionary
# master_edges = json.load(open('resources/Master_Edge_List_Dict.json', 'r'))

# read in relation data
relation_data = open('./resources/relations_data/RELATIONS_LABELS.txt').readlines()
relation_dict = {x.split('\t')[0]: x.split('\t')[1].strip('\n') for x in relation_data}

# print basic stats on each resource
edge_data = [[key,
              relation_dict[master_edges.source_info[key]['edge_relation']],
              ', '.join(master_edges.source_info[key]['edge_list'][0]),
              len(master_edges.source_info[key]['edge_list'])]
             for key in master_edges.source_info.keys()]

# convert dict to pandas df for nice printing
df = pandas.DataFrame(edge_data, columns = ['Edge Type', 'Relation', 'Example Edge', 'Unique Edges']) 
df                

<br><br>


***
***

## Build Knowledge Graph  <a class="anchor" id="build-kg"></a>
**Wiki Pages:**  
- **[`KG-Construction`](https://github.com/callahantiff/PheKnowLator/wiki/KG-Construction)**  
- **[`relations-data`](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#relations-data)**  
- **[`node-metadata`](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#node-metadata)**

<br>

**Assumptions:** That if relation and/or node metadata is going to be used to build the knowledge graph, that it has been generated and added to the `./resources/relations_data` and the `./resources/node_metadata` directories. Please see the [`Data_Preparation.ipynb`](https://github.com/callahantiff/PheKnowLator/blob/master/Data_Preparation.ipynb) Jupyter Notebook for details on how to create these data.  

<br>

**Input:** 
- [`Master_Edge_List_Dict.json`](https://www.dropbox.com/s/4j0vrwx26dh8hd1/Master_Edge_List_Dict.json?dl=1)  
- Directory of relations data sources - see [here](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#relations-data) for more information
- Directory of node data sources - see [here](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#node-metadata) for more information

<br>

**Output:** Please see [`Release v2.0.0 Wiki`](https://github.com/callahantiff/PheKnowLator/wiki/v2.0.0) for access to all generated output files.   
- `Knowledge Graph`  
- `Class Instance URI-UUID Map`  
- `Triple List - Integer`  
- `Triple List - Identifier`  
- `Node Integer-Identifier Map`  
- `Node Attribute Data`  

<br>

The process to build the knowledge graph is somewhat time consuming and can be broken into the following steps:  

1. Merge Ontologies  

2. Create `Class-Instance`, `Instance-Instance` and `Class-Class` Edges 

3. Add Inverse Relations and Node Data. See the [Dependencies](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies) Wiki page for details on how to construct these resources.  

4. Deductively Close Graph using [ELK](https://protegewiki.stanford.edu/wiki/ELK) Reasoner   
   - We recommend running this through the command line using [OWLTools](https://github.com/owlcollab/owltools). The code snippet below assumes that it has been downloaded to `./resources/lib` and demonstrates how the [ELK](https://www.cs.ox.ac.uk/isg/tools/ELK/) reasoner can be called and run over a knowledge graph saved in an `.owl` file.    

  ```bash
./resources/lib/owltools ./resources/knowledge_graphs/[input_graph_filename.owl] --reasoner elk --run-reasoner --assert-implied -o resources/knowledge_graphs/[output_graph_name]_Closed_ELK.owl
```

5. Filter OWL Semantics 
 - Filters the knowledge graph with the goal of removing all edges that contain entities that are needed to support owl semantics, but are not biologically meaningful. For example:
    - REMOVE - edge needed to support owl semantics that are not biologically meaningful:
      - subject: `http://purl.obolibrary.org/obo/CLO_0037294`
      - predicate: `owl:AnnotationProperty`
      - object: `rdf:about="http://purl.obolibrary.org/obo/CLO_0037294"`

    - KEEP - biologically meaningful edges:
      - subject: `http://purl.obolibrary.org/obo/CHEBI_16130`
      - predicate: `http://purl.obolibrary.org/obo/RO_0002606`
      - object: `http://purl.obolibrary.org/obo/HP_0000832`


6. Save Edge List    
 - Two versions of the knowledge graphs edges are saved as lists of triples one with: (1) node labels and (2) one with integer labels (the input requirement for the embedding algorithms), for example:

<br>

**‼ IMPORTANT:**  
- The file containing the merged ontologies is quite large and can take up to 30 minutes to read in.  This is not a limitation of the code directly, but rather a function of the [`RDFLib Library`](https://github.com/RDFLib). While there are other ways to read in this data, we maintain reliance on this library as it is the most user-friendly for non-RDF users.   
- If you'd like to include [node metadata](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#node-metadata) when building the knowledge graph, please hold off on building the knowledge graph until you have generated the node data. For details on how to do this see the [node metadata](https://github.com/callahantiff/PheKnowLator/wiki/Dependencies#node-metadata) section of the `Dependencies` Wiki Page or help for help with generating the data, please see the []() section of the [`Data_Preparation.ipynb`](https://github.com/callahantiff/PheKnowLator/blob/master/Data_Preparation.ipynb) Jupyter Notebook.

***


NEED TO ADD A DESCRIPTION HERE OF WHAT THE ARGUMENTS ARE

In [ ]:
# specify build type
build = 'full'

if build == 'partial':
    kg = pkt.PartialBuild(kg_version='v2.0.0',
                          write_location=args.out,
                          edge_data='./resources/Master_Edge_List_Dict.json',
                          node_data=args.nde,
                          inverse_relations=args.rel,
                          decode_owl_semantics=args.owl)
elif build == 'post-closure':
    kg = pkt.PostClosureBuild(kg_version='v2.0.0',
                              write_location=args.out,
                              edge_data='./resources/Master_Edge_List_Dict.json',
                              node_data=args.nde,
                              inverse_relations=args.rel,
                              remove_owl_semantics=args.owl)
else:
    kg = pkt.FullBuild(kg_version='v2.0.0',
                       write_location=args.out,
                       edge_data='./resources/Master_Edge_List_Dict.json',
                       node_data=args.nde,
                       inverse_relations=args.rel,
                       remove_owl_semantics=args.owl)

<br>

***
***

**TESTING:**

Only run chunks that do not remove OWL Semantics!

***
**RESULTS:** `Partial Node Metadata Relations Only OWL Semantics`
***

**Total Edges:** 20,565,453  
**Total Nodes:** 3,956,852  
**Run Time:** 4,007.616 seconds

In [ ]:
start = time.time()

kg = pkt.PartialBuild(kg_version='v2.0.0',
                          write_location=args.out,
                          edge_data='./resources/Master_Edge_List_Dict.json',
                          node_data=args.nde,
                          inverse_relations=args.rel,
                          decode_owl_semantics=args.owl)

kg.construct_knowledge_graph()

end = time.time()
print('Total time to run code: {} seconds'.format(end - start))

***
**RESULTS:** `Post-Closure Relations Only OWL Semantics`
***

**Total Edges:** xxx  
**Total Nodes:** xxx  
**Run Time:** xxx seconds

In [ ]:
start = time.time()

kg = pkt.PostClosureBuild(kg_version='v2.0.0',
                              write_location=args.out,
                              edge_data='./resources/Master_Edge_List_Dict.json',
                              node_data=args.nde,
                              inverse_relations=args.rel,
                              remove_owl_semantics=args.owl)

kg.construct_knowledge_graph()

end = time.time()
print('Total time to run code: {} seconds'.format(end - start))

***
**RESULTS:** `Post-Closure Relations Only No OWL Semantics`
***

**Total Edges:** xxx  
**Total Nodes:** xxx  
**Run Time:** xxx seconds

In [ ]:
start = time.time()

kg = pkt.PostClosureBuild(kg_version='v2.0.0',
                              write_location=args.out,
                              edge_data='./resources/Master_Edge_List_Dict.json',
                              node_data=args.nde,
                              inverse_relations=args.rel,
                              remove_owl_semantics=args.owl)

kg.construct_knowledge_graph()

end = time.time()
print('Total time to run code: {} seconds'.format(end - start))

<br><br><br><br>

***
**RESULTS:** `Partial Node Metadata Inverse Relations OWL Semantics`
***

**Total Edges:** 30,415,030  
**Total Nodes:** 3,956,852  
**Run Time:** 6,346.555 seconds

In [ ]:
start = time.time()

kg = pkt.PartialBuild(kg_version='v2.0.0',
                          write_location=args.out,
                          edge_data='./resources/Master_Edge_List_Dict.json',
                          node_data=args.nde,
                          inverse_relations=args.rel,
                          decode_owl_semantics=args.owl)

kg.construct_knowledge_graph()

end = time.time()
print('Total time to run code: {} seconds'.format(end - start))

***
**RESULTS:** `Post-Closure Inverse Relations OWL Semantics`
***

**Total Edges:** xxx  
**Total Nodes:** xxx  
**Run Time:** xxx seconds

In [ ]:
start = time.time()

kg = pkt.PostClosureBuild(kg_version='v2.0.0',
                              write_location=args.out,
                              edge_data='./resources/Master_Edge_List_Dict.json',
                              node_data=args.nde,
                              inverse_relations=args.rel,
                              remove_owl_semantics=args.owl)

kg.construct_knowledge_graph()

end = time.time()
print('Total time to run code: {} seconds'.format(end - start))

***
**RESULTS:** `Post-Closure Inverse Relations No OWL Semantics`
***

**Total Edges:** xxx  
**Total Nodes:** xxx  
**Run Time:** xxx seconds

In [ ]:
start = time.time()

kg = pkt.PostClosureBuild(kg_version='v2.0.0',
                              write_location=args.out,
                              edge_data='./resources/Master_Edge_List_Dict.json',
                              node_data=args.nde,
                              inverse_relations=args.rel,
                              remove_owl_semantics=args.owl)

kg.construct_knowledge_graph()

end = time.time()
print('Total time to run code: {} seconds'.format(end - start))

<br><br><br><br>

***
**RESULTS:** `Full - Relations Only OWL Semantics`
***

**Total Edges:** 21,856,388   
**Total Nodes:** 5,155,679   
**Run Time:** 7,398.384 seconds

In [ ]:
start = time.time()

kg = pkt.FullBuild(kg_version='v2.0.0',
                       write_location=args.out,
                       edge_data='./resources/Master_Edge_List_Dict.json',
                       node_data=args.nde,
                       inverse_relations=args.rel,
                       remove_owl_semantics=args.owl)

kg.construct_knowledge_graph()

end = time.time()
print('Total time to run code: {} seconds'.format(end - start))

***
**RESULTS:** `Full - Relations Only No OWL Semantics`
***

**Total Edges:** xxxx   
**Total Nodes:** xxxxx   
**Run Time:** xxxxx seconds

In [ ]:
start = time.time()

kg = pkt.FullBuild(kg_version='v2.0.0',
                       write_location=args.out,
                       edge_data='./resources/Master_Edge_List_Dict.json',
                       node_data=args.nde,
                       inverse_relations=args.rel,
                       remove_owl_semantics=args.owl)

kg.construct_knowledge_graph()

end = time.time()
print('Total time to run code: {} seconds'.format(end - start))

***
**RESULTS:** `Full - Inverse Relations OWL Semantics`
***

**Total Edges:** 31,705,965   
**Total Nodes:** 5,155,679   
**Run Time:** 11,667.708 seconds

In [ ]:
start = time.time()

kg = pkt.FullBuild(kg_version='v2.0.0',
                       write_location=args.out,
                       edge_data='./resources/Master_Edge_List_Dict.json',
                       node_data=args.nde,
                       inverse_relations=args.rel,
                       remove_owl_semantics=args.owl)

kg.construct_knowledge_graph()

end = time.time()
print('Total time to run code: {} seconds'.format(end - start))

***
**RESULTS:** `Full - Inverse Relations No OWL Semantics`
***

**Total Edges:** xxxx   
**Total Nodes:** xxxxx   
**Run Time:** xxxxx seconds

In [ ]:
start = time.time()

kg = pkt.FullBuild(kg_version='v2.0.0',
                       write_location=args.out,
                       edge_data='./resources/Master_Edge_List_Dict.json',
                       node_data=args.nde,
                       inverse_relations=args.rel,
                       remove_owl_semantics=args.owl)

kg.construct_knowledge_graph()

end = time.time()
print('Total time to run code: {} seconds'.format(end - start))

<br>

***
***

```
@misc{callahan_tj_2019_3401437,
  author       = {Callahan, TJ},
  title        = {PheKnowLator},
  month        = mar,
  year         = 2019,
  doi          = {10.5281/zenodo.3401437},
  url          = {https://doi.org/10.5281/zenodo.3401437}
}
```